In [1]:
# Import Splinter, Beautiful Soup, and Pandas
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [2]:
# Set up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/98.0.4758.102/chromedriver_mac64.zip
Driver has been saved in cache [/Users/kitwilliams/.wdm/drivers/chromedriver/mac64/98.0.4758.102]


# Scraping Mission to Mars Data

In [3]:
# Visit Mars News Site
url = 'https://redplanetscience.com'
browser.visit(url)
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

In [4]:
# Convert the browser HTML into soup object and then quit the browser
html = browser.html
news_soup = soup(html,'html.parser')
slide_elem = news_soup.select_one('div.list_text')

### Title and Summary

In [5]:
news_title = slide_elem.find('div',class_='content_title').get_text()
news_title

"NASA's Perseverance Rover 100 Days Out"

In [6]:
news_p = slide_elem.find('div',class_='article_teaser_body').get_text()
news_p

"Mark your calendars: The agency's latest rover has only about 8,640,000 seconds to go before it touches down on the Red Planet, becoming history's next Mars car."

### Featured Image

In [7]:
# Visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [8]:
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem

In [9]:
full_image_elem.click()

In [10]:
html = browser.html
img_soup = soup(html,'html.parser')

In [11]:
img_url_rel = img_soup.find('img',class_='fancybox-image').get('src')
img_url_rel

'image/featured/mars2.jpg'

In [12]:
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

'https://spaceimages-mars.com/image/featured/mars2.jpg'

### Mars Facts

In [13]:
df = pd.read_html('https://galaxyfacts-mars.com')[0]
df.columns=['description', 'Mars', 'Earth']
df.set_index('description', inplace=True)
df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [14]:
df.to_html();

# D1: Scrape High-Resolution Mars’ Hemisphere Images and Titles

In [15]:
# 1. Visit the URL
url = 'https://marshemispheres.com/'
browser.visit(url)

html = browser.html
soup_object = soup(html,'html.parser')

In [16]:
# 2. Create a list to hold images and titles
hemisphere_image_urls = []

In [17]:
# 3. Write code to retrieve the image urls and titles for each hemisphere.
results = soup_object.find('div',class_='collapsible results')
items = results.find_all('div',class_='item')

In [18]:
hemisphere_image_urls = [];

for item in items:
    hemispheres ={}
    
    link = item.find('a',class_='itemLink product-item').get('href')
    full_link = 'https://marshemispheres.com/'+link
    browser.visit(full_link)
    
    new_html = browser.html
    new_page_soup_object = soup(new_html,'html.parser')
    full_res = new_page_soup_object.find('div',class_='downloads').find('a').get('href')
    img_url = 'https://marshemispheres.com/'+full_res
    
    hemispheres['img_url'] = img_url
    
    img_name = new_page_soup_object.find('div',class_='cover').find('h2').text
    
    hemispheres['title'] = img_name
    hemisphere_image_urls.append(hemispheres)

In [19]:
# 4. Print the list that holds the dictionary of each image url and title.
hemisphere_image_urls

[{'img_url': 'https://marshemispheres.com/images/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [20]:
# 5. Quit the browser
browser.quit()